In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os

def intensity(phi, mask, slm_dim1, slm_dim2, sigma):
    # Calculate intensity at center pixel with noise
    mask2d = mask.reshape(slm_dim1, slm_dim2)
    field = np.exp(1j * 2 * np.pi * phi.reshape(slm_dim1, slm_dim2)) * mask2d
    spec = np.fft.fftshift(np.fft.fft2(field))
    I = np.abs(spec[slm_dim1 // 2, slm_dim2 // 2])**2 / spec.size
    I += sigma * np.random.randn()  # additive Gaussian noise
    return float(I)

def random_mask(slm_dim1, slm_dim2) -> np.ndarray:
    return np.random.choice([0.0, 1.0], size=slm_dim1*slm_dim2).astype(np.float32)

class ActorCriticGA(nn.Module):

    def __init__(self, input_dim, action_dim, hidden_dim=512):
        super().__init__()
        
        # Actor
        self.actor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, action_dim),
            nn.Sigmoid()
        )
        
        # Critic
        self.critic = nn.Sequential(
            nn.Linear(input_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 1)
        )

    def forward(self, state):
        if isinstance(state, np.ndarray):
            state = torch.FloatTensor(state)

        offspring_probs = self.actor(state)
        state_value = self.critic(state)
        return offspring_probs, state_value

    def generate_offspring(self, state, deterministic=False):
        offspring_probs, value = self.forward(state)
        dist = torch.distributions.Bernoulli(offspring_probs)

        if deterministic:
            offspring_mask = (offspring_probs > 0.5).float()
        else:
            offspring_mask = dist.sample()

        log_probs = dist.log_prob(offspring_mask)
        return offspring_mask, log_probs, value, offspring_probs

    def evaluate(self, states, actions):
        offspring_probs, state_values = self.forward(states)
        dist = torch.distributions.Bernoulli(offspring_probs)
        log_probs = dist.log_prob(actions).sum(dim=1)
        entropy = dist.entropy().sum(dim=1)
        return log_probs, state_values, entropy

    

class ACAgent:
    
    def __init__(self, input_dim, action_dim, actor_lr=0.001, critic_lr=0.001, batch_size=16, entropy_coef = 0.5):
        self.policy = ActorCriticGA(input_dim, action_dim)
        self.actor_optimizer = torch.optim.Adam(self.policy.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.policy.critic.parameters(), lr=critic_lr)
        self.batch_size = batch_size
        self.entropy_coef = entropy_coef  
        
        self.experience_buffer = {
            'states': [],
            'rewards': [],
            'log_probs': [],
            'values': [],
            'returns': [],
            'advantages': [],
            'children': []  
        }
        
        self.buffer_size = 1000  # Keep last N experiences
        self.min_buffer_size = 50  # Minimum experiences before training
    
    def generate_offspring(self, parent_state, deterministic=False):
        with torch.no_grad():
            if parent_state.dim() == 1:
                parent_state = parent_state.unsqueeze(0)

            offspring_mask, _, _, _ = self.policy.generate_offspring(parent_state, deterministic)
            
        return offspring_mask.squeeze(0).numpy().astype(np.float32)
    

    
    def store_experience(self, parent_state, offspring_fitness, child):
        # Add to experience buffer
        self.experience_buffer['states'].append(parent_state)
        self.experience_buffer['rewards'].append(offspring_fitness)
        self.experience_buffer['children'].append(child)
        
        # Generate log_probs and value for this state-action pair
        with torch.no_grad():
            parent_state_tensor = torch.FloatTensor(parent_state)
            child_tensor = torch.FloatTensor(child)
            
            offspring_probs, value = self.policy.forward(parent_state_tensor)
            dist = torch.distributions.Bernoulli(offspring_probs.squeeze())
            log_probs = dist.log_prob(child_tensor).sum()
            
            self.experience_buffer['log_probs'].append(log_probs.item())
            self.experience_buffer['values'].append(value.item())
        
        # Keep buffer size manageable
        if len(self.experience_buffer['states']) > self.buffer_size:
            for key in self.experience_buffer:
                self.experience_buffer[key] = self.experience_buffer[key][-self.buffer_size:]
    
    def compute_returns_and_advantages(self):
        # Compute returns and advantages for all experiences in buffer
        rewards = np.array(self.experience_buffer['rewards'])
        values = np.array(self.experience_buffer['values'])
        
        # Compute returns (copy rewards)
        returns = rewards.copy()
        
        # Normalize returns
        if returns.std() > 1e-8:
            returns = (returns - returns.mean()) / returns.std()
        
        # Compute advantages
        advantages = returns - values
        if advantages.std() > 1e-8:
            advantages = (advantages - advantages.mean()) / advantages.std()
        
        self.experience_buffer['returns'] = returns.tolist()
        self.experience_buffer['advantages'] = advantages.tolist()
    
    def get_training_batches(self, batch_size=None):
        if batch_size is None:
            batch_size = self.batch_size

        n_samples = len(self.experience_buffer['states'])
        indices = np.random.permutation(n_samples)

        batches = []
        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            batch_indices = indices[start:end]
            batch = {key: [self.experience_buffer[key][i] for i in batch_indices]
                    for key in self.experience_buffer}
            batches.append(batch)

        return batches

    def update_policy(self, n_epochs=10):
        if len(self.experience_buffer['states']) < self.min_buffer_size:
            return None, None

        # Compute returns and advantages for all experiences
        self.compute_returns_and_advantages()

        total_actor_loss = 0
        total_critic_loss = 0
        total_entropy_loss = 0
        total_policy_loss = 0
        n_batches = 0


        for epoch in range(n_epochs):
            for batch in self.get_training_batches():
                states = torch.FloatTensor(batch['states'])
                returns = torch.FloatTensor(batch['returns'])
                advantages = torch.FloatTensor(batch['advantages'])
                children = torch.FloatTensor(batch['children'])

                log_probs, state_values, entropy = self.policy.evaluate(states, children)

                # Actor loss
                policy_loss = -(log_probs * advantages).mean()
                entropy_loss = -entropy.mean()  # Encourage exploration
                actor_loss = policy_loss + self.entropy_coef * entropy_loss

                # Critic loss
                critic_loss = F.smooth_l1_loss(state_values.squeeze(-1), returns)

                # Update networks
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.policy.actor.parameters(), 0.5)
                self.actor_optimizer.step()

                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.policy.critic.parameters(), 0.5)
                self.critic_optimizer.step()

                total_actor_loss += actor_loss.item()
                total_critic_loss += critic_loss.item()
                total_entropy_loss += entropy_loss.item()
                total_policy_loss += policy_loss.item()
                n_batches += 1

        avg_actor_loss = total_actor_loss / n_batches
        avg_critic_loss = total_critic_loss / n_batches
        avg_entropy_loss = total_entropy_loss / n_batches
        avg_policy_loss = total_policy_loss / n_batches

        self.entropy_coef = max(0.1, self.entropy_coef * 0.9)

        return avg_actor_loss, avg_critic_loss, avg_entropy_loss, avg_policy_loss
    
    def update_policy_intensive(self, n_epochs=10):
        # Intensive training for early generations to stabilize networks
        return self.update_policy(n_epochs=n_epochs)


def binary_ga(slm_dim1: int = 16,
              slm_dim2: int = 16,
              pop_size: int = 30,
              generations: int = 200,
              sigma: float = 0.0,
              phi: np.ndarray | None = None,
              seed: int | None = None,
              update_freq: int = 5,
              stabilization_gens: int = 20,
              intensive_epochs: int = 100,
              normal_epochs: int = 10):

    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)

    n_pix = slm_dim1 * slm_dim2
    if phi is None:
        phi = np.random.rand(n_pix).astype(np.float32)

    # Initialize parent population
    population = [random_mask(slm_dim1, slm_dim2) for _ in range(pop_size)]
    best_int_history = []

    # Initialize AC agent
    agent = ACAgent(
        input_dim=n_pix * 2 + 2, 
        action_dim=n_pix, 
        actor_lr=0.00009, 
        critic_lr=0.00009, 
        batch_size=16, 
        entropy_coef=0.5
    )

    print(f"Starting Binary GA with AC integration...")
    print(f"Population size: {pop_size}, Generations: {generations}")
    print(f"SLM dimensions: {slm_dim1}x{slm_dim2}, Pixels: {n_pix}")
    print(f"Network stabilization: {stabilization_gens} gens with {intensive_epochs} epochs")
    print(f"Normal training: {normal_epochs} epochs every {update_freq} generations")

    policy_losses = []
    actor_losses = []
    critic_losses = []
    entropy_losses = []
    eval_mean_intensities = []
    eval_max_intensities = []

    for g in range(1, generations + 1):
        # 1) Evaluate current population
        intensities = [intensity(phi, mask, slm_dim1, slm_dim2, sigma) for mask in population]
        order = np.argsort(intensities)[::-1]  # Descending order

        # 2) Generate offspring using AC policy
        offspring = []
        generation_experiences = []
        
        for _ in range(pop_size):
            # Select two parents from top half
            top_half_size = pop_size // 2
            idx_pool = np.random.choice(top_half_size, size=2, replace=False)
            
            parent1_idx = order[idx_pool[0]]
            parent2_idx = order[idx_pool[1]]
            
            parent1 = population[parent1_idx]
            parent2 = population[parent2_idx]
            score1 = intensities[parent1_idx]
            score2 = intensities[parent2_idx]
            
            # Generate offspring using AC policy
            parent_state = np.concatenate([parent1, parent2, [score1], [score2]])
            parent_state_tensor = torch.FloatTensor(parent_state)
            child = agent.generate_offspring(parent_state_tensor, deterministic=False)
            offspring.append(child)
            
            # Store parent info for later training
            generation_experiences.append((parent_state, child))

        # 3) Evaluate offspring
        offspring_intensities = [intensity(phi, mask, slm_dim1, slm_dim2, sigma) for mask in offspring]

        # 4) Store experiences for AC training
        for i, (parent_state, child) in enumerate(generation_experiences):
            offspring_fitness = offspring_intensities[i]
            agent.store_experience(parent_state, offspring_fitness, child)

        # 5) Replace worst half of population with best offspring
        offspring_order = np.argsort(offspring_intensities)[::-1]
        worst_half_size = pop_size // 2
        for k in range(worst_half_size):
            worst_parent_idx = order[-(k + 1)]  # Start from worst
            best_offspring_idx = offspring_order[k]  # Best offspring
            population[worst_parent_idx] = offspring[best_offspring_idx]

        # 6) Update AC policy with appropriate training intensity
        if g % update_freq == 0:
            # Intensive training for early generations to stabilize networks
            if g <= stabilization_gens:
                actor_loss, critic_loss, entropy_loss, policy_loss = agent.update_policy_intensive(intensive_epochs)
                training_type = "Intensive"
            else:
                actor_loss, critic_loss, entropy_loss, policy_loss = agent.update_policy(normal_epochs)
                training_type = "Normal"
            
            eval_intensities, eval_mean_intensity = evaluate_agent_deterministic(agent, population, phi, slm_dim1, slm_dim2, sigma=0.0)
            eval_max_intensities.append(max(eval_intensities))
            eval_mean_intensities.append(eval_mean_intensity)


            if actor_loss is not None and critic_loss is not None:
                print(f"[{training_type} AC Update] Gen {g}: Actor Loss = {actor_loss:.4f}, Critic Loss = {critic_loss:.4f}")
                policy_losses.append(policy_loss)
                actor_losses.append(actor_loss)
                critic_losses.append(critic_loss)
                entropy_losses.append(entropy_loss)
            else:
                print(f"[AC Update] Gen {g}: Insufficient data for training")

        # 7) Track best fitness
        current_intensities = [intensity(phi, mask, slm_dim1, slm_dim2, sigma) for mask in population]
        best_current = max(current_intensities)
        best_int_history.append(best_current)

       
        print(f"Gen {g:3d}  best = {best_current:.6f}")

    return {'max_intensity': best_int_history,
            'policy_losses': policy_losses,
            'actor_losses': actor_losses,
            'critic_losses': critic_losses,
            'entropy_losses': entropy_losses,
            'eval_mean_intensities': eval_mean_intensities,
            'eval_max_intensities': eval_max_intensities}

def evaluate_agent_deterministic(agent, population, phi, slm_dim1, slm_dim2, sigma):
    # Evaluate current policy network deterministically on given population.
    pop_size = len(population)

    # 1. Evaluate fitness of current parents
    parent_intensities = [intensity(phi, mask, slm_dim1, slm_dim2, sigma) for mask in population]
    order = np.argsort(parent_intensities)[::-1]

    deterministic_offspring = []
    for _ in range(pop_size):
        # Choose parents from top half
        top_half = pop_size // 2
        p1_idx, p2_idx = np.random.choice(top_half, size=2, replace=False)
        idx1, idx2 = order[p1_idx], order[p2_idx]

        parent1 = population[idx1]
        parent2 = population[idx2]
        score1 = parent_intensities[idx1]
        score2 = parent_intensities[idx2]

        # Construct input state
        state = np.concatenate([parent1, parent2, [score1], [score2]])
        state_tensor = torch.FloatTensor(state)

        # Generate deterministic offspring
        child = agent.generate_offspring(state_tensor, deterministic=True)
        deterministic_offspring.append(child)

    # 2. Evaluate intensities
    eval_intensities = [intensity(phi, child, slm_dim1, slm_dim2, sigma) for child in deterministic_offspring]
    mean_eval_intensity = np.mean(eval_intensities)

    return eval_intensities, mean_eval_intensity

def plot_ga_ac_training(results, run_id='default'):
    plt.figure(figsize=(16, 16))

    # 1. Best Intensity Curve
    if 'max_intensity' in results and len(results['max_intensity']) > 0:
        plt.subplot(7, 1, 1)
        plt.plot(results['max_intensity'], label='Best Intensity during Training', color='black')
        plt.xlabel('Generation')
        plt.ylabel('Intensity')
        plt.title('Best Intensity Per Generation')
        plt.grid(True)
        plt.legend()

    # 2. Actor Loss Trend
    if 'actor_losses' in results and len(results['actor_losses']) > 0:
        plt.subplot(7, 1, 2)
        plt.plot(results['actor_losses'], label='Actor Loss', color='blue')
        plt.xlabel('Update Step')
        plt.ylabel('Loss')
        plt.title('Actor Loss Over Updates')
        plt.grid(True)
        plt.legend()

    # 3. Critic Loss Trend
    if 'critic_losses' in results and len(results['critic_losses']) > 0:
        plt.subplot(7, 1, 3)
        plt.plot(results['critic_losses'], label='Critic Loss', color='red')
        plt.xlabel('Update Step')
        plt.ylabel('Loss')
        plt.title('Critic Loss Over Updates')
        plt.grid(True)
        plt.legend()

    # 4. Policy Gradient Loss (before entropy term)
    if 'policy_losses' in results and len(results['policy_losses']) > 0:
        plt.subplot(7, 1, 4)
        plt.plot(results['policy_losses'], label='Policy Gradient Loss', color='green')
        plt.xlabel('Update Step')
        plt.ylabel('Loss')
        plt.title('Policy Gradient Loss')
        plt.grid(True)
        plt.legend()

    # 5. Entropy Loss
    if 'entropy_losses' in results and len(results['entropy_losses']) > 0:
        plt.subplot(7, 1, 5)
        plt.plot(results['entropy_losses'], label='Entropy Loss', color='purple')
        plt.xlabel('Update Step')
        plt.ylabel('Entropy')
        plt.title('Entropy Loss Over Updates')
        plt.grid(True)
        plt.legend()
    
    # 6. Evaluation Mean Intensities
    if 'eval_mean_intensities' in results and len(results['eval_mean_intensities']) > 0:
        plt.subplot(7, 1, 6)
        plt.plot(results['eval_mean_intensities'], label='Mean Intensity (Eval)', color='orange')
        plt.xlabel('Update Step')
        plt.ylabel('Mean Intensity')
        plt.title('Mean Intensity of Offspring (Evaluation)')
        plt.grid(True)
        plt.legend()
    
    # 7. Evaluation Max Intensities
    if 'eval_max_intensities' in results and len(results['eval_max_intensities']) > 0:
        plt.subplot(7, 1, 7)
        plt.plot(results['eval_max_intensities'], label='Max Intensity (Eval)', color='cyan')
        plt.xlabel('Update Step')
        plt.ylabel('Max Intensity')
        plt.title('Max Intensity of Offspring (Evaluation)')
        plt.grid(True)
        plt.legend()


    plt.tight_layout()
    plt.savefig(f'results_ga_rl/run_{run_id}_ga_ac_training_curve.png')
    plt.close()


def main(run=0):

    np.random.seed(42)
    torch.manual_seed(42)

    phi_path = "phase_mask/phi_16.npy"

    if os.path.exists(phi_path):
        phi = np.load(phi_path)
    else:
        phi = np.random.rand(16, 16).astype(np.float32)
        os.makedirs(os.path.dirname(phi_path), exist_ok=True)
        np.save(phi_path, phi)


    result = binary_ga(
        slm_dim1=16, 
        slm_dim2=16, 
        pop_size=900, 
        generations=8,
        sigma=0.01,
        phi=phi,
        seed=42,
        update_freq=1,              
        stabilization_gens=1,      
        intensive_epochs=1,        
        normal_epochs=1,           
    )

    # Plot the training results
    plot_ga_ac_training(result, run_id=run)

    history = result['max_intensity']
    print(f"\nFinal best intensity: {max(history):.6f}")
    print(f"Improvement: {(max(history) - history[0])/history[0]*100:.2f}%")


main(0)

Starting Binary GA with AC integration...
Population size: 900, Generations: 8
SLM dimensions: 16x16, Pixels: 256
Network stabilization: 1 gens with 1 epochs
Normal training: 1 epochs every 1 generations


C:\Users\SUJATHA\AppData\Local\Temp\ipykernel_11184\1250685451.py:182: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  states = torch.FloatTensor(batch['states'])


[Intensive AC Update] Gen 1: Actor Loss = -88.1438, Critic Loss = 0.3720
Gen   1  best = 2.725623
[Normal AC Update] Gen 2: Actor Loss = -79.6213, Critic Loss = 0.3862
Gen   2  best = 2.814578
[Normal AC Update] Gen 3: Actor Loss = -72.6002, Critic Loss = 0.3898
Gen   3  best = 3.779778
[Normal AC Update] Gen 4: Actor Loss = -65.7730, Critic Loss = 0.4026
Gen   4  best = 5.232479
[Normal AC Update] Gen 5: Actor Loss = -60.3191, Critic Loss = 0.4155
Gen   5  best = 5.234857
[Normal AC Update] Gen 6: Actor Loss = -52.5538, Critic Loss = 0.4205
Gen   6  best = 5.774911
[Normal AC Update] Gen 7: Actor Loss = -48.7117, Critic Loss = 0.4186
Gen   7  best = 6.570865
[Normal AC Update] Gen 8: Actor Loss = -44.4934, Critic Loss = 0.4230
Gen   8  best = 8.121511

Final best intensity: 8.121511
Improvement: 197.97%


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

def intensity(phi, mask, slm_dim1, slm_dim2, sigma):
    mask2d = mask.reshape(slm_dim1, slm_dim2)
    field = np.exp(1j * 2 * np.pi * phi.reshape(slm_dim1, slm_dim2)) * mask2d
    spec = np.fft.fftshift(np.fft.fft2(field))
    I = np.abs(spec[slm_dim1 // 2, slm_dim2 // 2])**2 / spec.size
    I += sigma * np.random.randn()  # additive Gaussian noise
    return float(I)

def random_mask(slm_dim1, slm_dim2) -> np.ndarray:
    return np.random.choice([0.0, 1.0], size=slm_dim1*slm_dim2).astype(np.float32)

def binary_ga(slm_dim1: int = 16,
              slm_dim2: int = 16,
              pop_size: int = 30,
              generations: int = 200,
              sigma: float = 0.0,
              R0: float = 0.1,
              Rend: float = 0.0025,
              lambda_: float = 50.0,
              phi: np.ndarray | None = None,
              seed: int | None = None):
    
    if seed is not None:
        np.random.seed(seed)

    n_pix = slm_dim1 * slm_dim2
    if phi is None:
        phi = np.random.rand(n_pix).astype(np.float32)

    population = [random_mask(slm_dim1, slm_dim2) for _ in range(pop_size)]
    best_int_history = []
    best_mask = None

    for g in range(1, generations + 1):
        # 1) Evaluate current parents
        intensities = [intensity(phi, m, slm_dim1, slm_dim2, sigma) for m in population]
        order = np.argsort(intensities)[::-1]  # Descending

        # 2) Crossover template
        T_flat = np.zeros(n_pix, dtype=np.float32)
        T_flat[np.random.choice(n_pix, n_pix // 2, replace=False)] = 1.0
        T_breed = T_flat.reshape(slm_dim1, slm_dim2)

        # 3) Generate offspring via crossover
        offspring = []
        for _ in range(pop_size):
            idx_pool = np.random.permutation(pop_size // 2)[:2]
            pa = population[order[idx_pool[0]]].reshape(slm_dim1, slm_dim2)
            ma = population[order[idx_pool[1]]].reshape(slm_dim1, slm_dim2)
            child = pa * T_breed + ma * (1 - T_breed)
            offspring.append(child)

        # 4) Mutate offspring with decaying rate
        mut_rate = (R0 - Rend) * math.exp(-g / lambda_) + Rend
        mut_num = int(round(mut_rate * n_pix))
        mutated = []
        for child in offspring:
            flat = child.flatten()
            flip_idx = np.random.choice(n_pix, mut_num, replace=False)
            flat[flip_idx] = 1.0 - flat[flip_idx]  # Bit‑flip mutation
            mutated.append(flat.reshape(slm_dim1, slm_dim2))

        # 5) Evaluate offspring
        off_int = [intensity(phi, m, slm_dim1, slm_dim2, sigma) for m in mutated]
        off_order = np.argsort(off_int)[::-1]

        # 6) Replacement
        for k in range(pop_size // 2):
            population[order[-(k + 1)]] = mutated[off_order[k]]

        # 7) Logging
        best_parent_int = intensities[order[0]]
        best_off_int = off_int[off_order[0]]
        best_int = max(best_parent_int, best_off_int)

        if best_int == best_parent_int:
            best_mask = population[order[0]]
        else:
            best_mask = mutated[off_order[0]]

        best_int_history.append(best_int)
        print(f"Gen {g:3d}  Best Intensity = {best_int:.4f}")

    return {
        'max_intensity': best_int_history,
        'best_mask': best_mask
    }

phi = np.load("phase_mask/phi.npy")
results = binary_ga(slm_dim1=16, slm_dim2=16, pop_size=30, generations=200, sigma=0.01, seed=42, phi=phi)
print(f"Final best intensity: {max(results['max_intensity']):.6f}")

plt.figure(figsize=(16, 16))
plt.plot(results['max_intensity'], label='Best Intensity during Training', color='black')
plt.xlabel('Generation')
plt.ylabel('Intensity')
plt.title('Best Intensity Per Generation')
plt.grid(True)
plt.legend()
plt.savefig('results_ga/binary_ga_training_curve.png')
plt.close()

[Binary-GA] Gen   1  Best Intensity = 1.6213
[Binary-GA] Gen   2  Best Intensity = 2.6183
[Binary-GA] Gen   3  Best Intensity = 2.6242
[Binary-GA] Gen   4  Best Intensity = 2.6486
[Binary-GA] Gen   5  Best Intensity = 3.1527
[Binary-GA] Gen   6  Best Intensity = 3.1477
[Binary-GA] Gen   7  Best Intensity = 4.4219
[Binary-GA] Gen   8  Best Intensity = 5.3945
[Binary-GA] Gen   9  Best Intensity = 5.3906
[Binary-GA] Gen  10  Best Intensity = 5.3928
[Binary-GA] Gen  11  Best Intensity = 5.4149
[Binary-GA] Gen  12  Best Intensity = 5.6456
[Binary-GA] Gen  13  Best Intensity = 5.7987
[Binary-GA] Gen  14  Best Intensity = 6.0462
[Binary-GA] Gen  15  Best Intensity = 6.9040
[Binary-GA] Gen  16  Best Intensity = 6.9116
[Binary-GA] Gen  17  Best Intensity = 6.9019
[Binary-GA] Gen  18  Best Intensity = 6.9183
[Binary-GA] Gen  19  Best Intensity = 7.4282
[Binary-GA] Gen  20  Best Intensity = 8.7649
[Binary-GA] Gen  21  Best Intensity = 8.7602
[Binary-GA] Gen  22  Best Intensity = 8.7675
[Binary-GA